# ⚛️ H₃⁺ Noisy VQE — UCC-S vs UCC-D vs UCCSD

This notebook benchmarks **noisy VQE** for the trihydrogen cation **H₃⁺**
using three chemistry-inspired ansatzes implemented in your package:

- **UCC-S** — singles only  
- **UCC-D** — doubles only  
- **UCCSD** — singles + doubles  

All of the following are handled entirely by the package:

- Molecular Hamiltonian construction  
- Ansatz construction (UCC-S / UCC-D / UCCSD)  
- Device selection (`default.mixed` for noisy runs)  
- Noise channels (depolarizing + amplitude damping)  
- Caching & reproducibility  

via the single high-level entrypoint:

```python
from vqe.core import run_vqe
```

This notebook is therefore a **pure client** of the VQE package — no manual
QNodes, devices, or excitations are defined here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Allow the notebook to import the local package when run from the repo
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.core import run_vqe
from vqe_qpe_common.molecules import get_molecule_config
from vqe_qpe_common.plotting import IMG_DIR

# 🔧 Molecular System & Noise Model

We load the **H₃⁺** configuration from the shared molecule registry and then
fix a simple NISQ-style noise model:

- **Depolarizing noise:** 2% per qubit  
- **Amplitude damping:** 4% per qubit  

These values are intentionally a bit exaggerated so that the effect of noise
on different ansatzes is clearly visible.

In [ ]:
# Load H₃⁺ from the shared registry (used elsewhere in the project)
mol = get_molecule_config("H3+")
print("Loaded H₃⁺ configuration:")
for k, v in mol.items():
    print(f"  {k}: {v}")

# Noise parameters (per qubit)
depolarizing_prob = 0.02
amplitude_damping_prob = 0.04

# Ansätze to compare
ansatzes = ["UCC-S", "UCC-D", "UCCSD"]
results = {}

# Fixed optimisation / run settings
optimizer_name = "Adam"
n_steps = 50
stepsize = 0.2
seed = 0

# 🚀 Run Noisy VQE for Each Ansatz

For each ansatz in:

- `"UCC-S"`, `"UCC-D"`, `"UCCSD"`

we call:

```python
run_vqe(
    molecule="H3+",
    ansatz_name=...,
    optimizer_name="Adam",
    n_steps=50,
    stepsize=0.2,
    noisy=True,
    depolarizing_prob=0.02,
    amplitude_damping_prob=0.04,
    seed=0,
)
```

`run_vqe`:

- builds the Hamiltonian and ansatz  
- creates a noisy `default.mixed` device  
- runs the optimisation loop  
- returns energies and the final (noisy) density matrix.

In [ ]:
for ans in ansatzes:
    print(f"\n▶ Running NOISY VQE for H₃⁺ with ansatz: {ans}")
    res = run_vqe(
        molecule="H3+",
        ansatz_name=ans,
        optimizer_name=optimizer_name,
        n_steps=n_steps,
        stepsize=stepsize,
        noisy=True,
        depolarizing_prob=depolarizing_prob,
        amplitude_damping_prob=amplitude_damping_prob,
        seed=seed,
        plot=False,
        force=False,
    )
    results[ans] = res

# 📉 Convergence Comparison

We now plot **energy vs iteration** for each ansatz under the same noisy
conditions. This highlights:

- how quickly each ansatz converges  
- whether they get trapped at different noisy optima  
- the relative robustness of UCC-S / UCC-D / UCCSD to this noise model

In [ ]:
plt.figure(figsize=(10, 6))
for ans in ansatzes:
    plt.plot(results[ans]["energies"], label=ans)

plt.title("H₃⁺ Noisy VQE — UCC-S vs UCC-D vs UCCSD")
plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.legend()
plt.grid(True, alpha=0.4)
plt.tight_layout()

fname = os.path.join(IMG_DIR, "H3plus_Noisy_Ansatz_Comparison.png")
plt.savefig(fname, dpi=300)
plt.show()

print(f"\n📁 Saved convergence plot to: {fname}")

# 📊 Final Energies Summary

A quick table of the final energies for each ansatz:

In [ ]:
print("\nFinal energies (noisy runs):")
for ans in ansatzes:
    print(f"  {ans:6s} → E = {results[ans]['energy']:.8f} Ha")

# 🔬 Approximate Population Distributions

In the **noisy** setting (`default.mixed`), `run_vqe` stores the final
**density matrix** ρ via its real and imaginary components:

- `final_state_real`  
- `final_state_imag`

For intuition, we look at the **diagonal** of ρ:

$$
p_i = \rho_{ii}
$$

which gives the population of each computational basis state $|i⟩$.

We construct a compact, human-readable summary:

$$
\{p_i\}_i \quad \Rightarrow \quad \sum_i p_i |i⟩
$$

(This is **not** a true pure-state ket; it is just a convenient way
to report the diagonal populations.)

In [ ]:
def approx_population(record, threshold: float = 1e-2) -> str:
    """
    Build a human-readable population summary from the stored state.

    Works for both:
      - pure states (statevector) → use |ψ_i|²
      - mixed states (density matrix) → use diag(ρ)
    """
    r = np.array(record["final_state_real"])
    i = np.array(record["final_state_imag"])
    state = r + 1j * i

    # Pure statevector case
    if state.ndim == 1:
        diag = np.abs(state) ** 2
    # Density matrix case
    elif state.ndim == 2:
        diag = np.real(np.diag(state))
    else:
        raise ValueError(f"Unexpected state shape: {state.shape}")

    n = int(np.log2(len(diag)))

    terms = [
        f"({p:.4f})|{idx:0{n}b}⟩"
        for idx, p in enumerate(diag)
        if p > threshold
    ]

    return " + ".join(terms) if terms else "(all populations below threshold)"

In [ ]:
for ans in ansatzes:
    print("\n" + "-" * 70)
    print(f"Approximate population distribution for {ans}:")
    print(approx_population(results[ans], threshold=1e-2))